# 전처리

## 크롤링 파일 로드

In [1]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('[KB]아싸_일반_전처리.csv')

In [3]:
#중복제거

df = df.drop_duplicates(['body'], keep='first')
df = df.reset_index(drop = True)
len(df)

12358

In [4]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    return text

In [5]:
df['body'] = df['body'].apply(preprocess_text)
df['title'] = df['title'].apply(preprocess_text)
df['comment'] = df['comment'].apply(preprocess_text)
df['date'] = df['date'].apply(preprocess_text)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12358 entries, 0 to 12357
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  12358 non-null  int64 
 1   keyword     12358 non-null  object
 2   board       12358 non-null  object
 3   title       12351 non-null  object
 4   body        12357 non-null  object
 5   comment     12358 non-null  object
 6   date        12358 non-null  object
 7   review      12358 non-null  object
dtypes: int64(1), object(7)
memory usage: 772.5+ KB


## 제목, 본문, 댓글을 하나로 통합

In [7]:
# 혹시 리뷰 중에 Null 값을 가진 샘플이 있는지 확인
print(df.isnull().sum())

Unnamed: 0    0
keyword       0
board         0
title         7
body          1
comment       0
date          0
review        0
dtype: int64


In [8]:
# 위에서 Null값이 나오면 공백으로 수정 (Null 값 포함해서 doc로 합치면 Null값으로 반환됨)
df['body'] = df['body'].fillna('')

print(df.isnull().values.any()) # 제거되었는지 확인

True


In [9]:
# 제목, 본문, 댓글 합쳐 하나의 document로 만들기

df.loc[:,'review'] = df['title'] + ' ' + df['body'] + ' ' + df['comment']

In [10]:
df = df.reset_index(drop=True)

In [11]:
df

,Unnamed: 0,keyword,board,title,body,comment,date,review
0,0,절세,• 일반소매점·편의점 Talk,편의점 마트 전기세,두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스 나신분들도 많을듯...,기존에 80냈었으니까 두배이상 뛰었으면......... 다행히(?) 90냈습니다. ...,2023.02.14.,편의점 마트 전기세 두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스...
1,1,절세,• 일반소매점·편의점 Talk,5월 종합소득세 신고에서요,제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙으로 인정 되나요? 지금이라...,사업자 카드아니여도 홈택스에 등록하시면 됩니다. 네 정보 감사합니다. 잊고 있었네요...,2023.02.10.,5월 종합소득세 신고에서요 제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙...
2,2,절세,• 일반소매점·편의점 Talk,사장님들 저가 sns로 소매로 판매하고있는데용,저가 초보라 잘몰라용 가루쳐주세요 ㅜㅜ 사업자 등록 ㅊ통신판매업 등록은 완료했는데 ...,부가세 상반기 하반기 신고 하시고 종소세도 있어요 영수증 발행으로 매출이 잡히니까 ...,2022.09.16.,사장님들 저가 sns로 소매로 판매하고있는데용 저가 초보라 잘몰라용 가루쳐주세요 ㅜ...
3,3,절세,• 일반소매점·편의점 Talk,가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...,10만원에 부가세 별도라고 하던데요.. 인건비도 따로 나가는게 없는데 좀 비싼것 같...,가격은 천차만별이라.. 잘해주는 곳이랑 하는게 제일 좋죠;보통 8~10마넌 부가세별...,2021.12.28.,가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...
4,4,절세,• 일반소매점·편의점 Talk,편의점 새내기입니다 선배님들께 여쭤봅니다!,아싸카페에서 인정받은 유일한 마케팅회사 http://www.reviewnote.kr...,쓰면 나쁠건 없어요 꼭 써야하는건 아니구요. 조언감사합니당. 저도 바로 전화와서 현...,2021.11.17.,편의점 새내기입니다 선배님들께 여쭤봅니다! 아싸카페에서 인정받은 유일한 마케팅회사 ...
...,...,...,...,...,...,...,...,...
12353,12375,반품,• 일반소매점·편의점 Talk,팔기싫은 손님 ㅠ,저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요 증~말 팔기싫은 ...,"힘드시겠어요,...... 생각만 해도 아,. 네~그게 다에요~네~다른 사이즈 없어요...",2021.04.05.,팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354,12376,반품,• 일반소매점·편의점 Talk,담배반품 언제까지 받아주시나요?,구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요. 담배를 끊었다나 뭐라나... ...,법적으로 한달 아닌가요..... 저도 이건궁굼하네요 저런상황에서 해줘야하는건가요.....,2020.06.09.,담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355,12377,반품,• 일반소매점·편의점 Talk,반품 할 것들은요,그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 다시 빼야하는데 그...,반품전표랑 같이 박스에 두심되요. 반품전표란같이나두세여.,2020.04.17.,반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356,12378,반품,• 일반소매점·편의점 Talk,유통기한지난식품판매,안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되었네요.. 사과도 했...,합의하세요. 돈을주시는게낳아요..벌금금액이더큽니다.,2020.01.28.,유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...


In [12]:
df_doc = df.review # document 로 합친 부분만 새로운 변수에 저장해준다.

In [13]:
df_doc[250:]

250      간이과세자 월세 올해 창업한 간이사업자인데 월세 납부를 부가세없이 내는것이 좋을까요...
251      편의점배분율궁금 사장님들ㅡ궁금해서물어봅니다 편의점본사와의비율이 7대3,8대2 이뜻은...
252      비씨카드 고객센터 상담원 연결이 안 되요ㅠㅠㅠㅠ 방법이 없을까요?? CCTV로 여자...
253      정육 이랑 아이스크림 아싸카페에서 인정받은 유일한 마케팅회사 http://www.r...
254      방역지원금 부지급 아싸카페에서 인정받은 유일한 마케팅회사 http://www.rev...
                               ...                        
12353    팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354    담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355    반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356    유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...
12357    편돌이사장님 질문보고 저도 퇴근전 문득 질문이요~ 이제 3년차 접어들은 소형슈퍼 새...
Name: review, Length: 12108, dtype: object

## 광고글을 삭제하기 위해 특정 단어가 들어간 행을 전체 삭제한다

In [14]:
words_to_delete = ['[기사]', '(웃픈글)']
pattern = '|'.join(words_to_delete)

delete = df['review'].str.contains(pattern)

C:\Users\tinac\AppData\Local\Temp\ipykernel_8200\1058786048.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  delete = df['review'].str.contains(pattern)


In [ ]:
df['review'][df.index[delete]] #삭제할 행 확인

In [ ]:
df_doc.drop(df_doc.index[delete], inplace = True)
df_doc = df_doc.reset_index(drop=True)
df_doc

### 반복되는 문장은 삭제해준다. (잘못된 분석 결과를 야기할 수 있기 때문에)
- 이는 크롤링 대상 카페/게시판/블로그/커뮤니티 마다 다르기 때문에 크롤링한 텍스트를 보고 사전에 파악하는 것이 좋다

In [17]:
# 따옴표 안에 삭제할 문구를 복사해서 붙여넣기 해준다.

df_doc = df_doc.str.replace("편의점 운영 노하우 일기","")
df_doc = df_doc.str.replace("하루 3가지 편의점 일기","")
df_doc = df_doc.str.replace("아싸카페에서 인정받은 유일한 마케팅회사 http://www.reviewnote.kr","")


In [18]:
df_doc

0        편의점 마트 전기세 두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스...
1        5월 종합소득세 신고에서요 제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙...
2        사장님들 저가 sns로 소매로 판매하고있는데용 저가 초보라 잘몰라용 가루쳐주세요 ㅜ...
3        가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...
4        편의점 새내기입니다 선배님들께 여쭤봅니다!  이달25일부터 c사 편의점 인수예정인데...
                               ...                        
12353    팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354    담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355    반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356    유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...
12357    편돌이사장님 질문보고 저도 퇴근전 문득 질문이요~ 이제 3년차 접어들은 소형슈퍼 새...
Name: review, Length: 12358, dtype: object

### 텍스트 전처리
- 분석가의 판단 하에 문장부호, 특수문자, 숫자, 영어, 이모티콘 등 삭제

In [19]:
df_doc = df_doc.str.replace("[0-9]+갤","개월") # ex.8갤->개월(맞춤법 검사 시 교정 안 됌)
df_doc = df_doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
df_doc = df_doc.str.replace("\s+", " ") # white space 제거

In [20]:
df_doc

0        편의점 마트 전기세 두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스...
1        5월 종합소득세 신고에서요 제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙...
2        사장님들 저가 sns로 소매로 판매하고있는데용 저가 초보라 잘몰라용 가루쳐주세요 ㅜ...
3        가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...
4        편의점 새내기입니다 선배님들께 여쭤봅니다!  이달25일부터 c사 편의점 인수예정인데...
                               ...                        
12353    팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354    담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355    반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356    유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...
12357    편돌이사장님 질문보고 저도 퇴근전 문득 질문이요~ 이제 3년차 접어들은 소형슈퍼 새...
Name: review, Length: 12358, dtype: object

In [21]:
doc = pd.DataFrame(df_doc) # 위에서 전처리한 것을 dataframe으로 변환시켜 준다.

In [22]:
doc

,review
0,편의점 마트 전기세 두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스...
1,5월 종합소득세 신고에서요 제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙...
2,사장님들 저가 sns로 소매로 판매하고있는데용 저가 초보라 잘몰라용 가루쳐주세요 ㅜ...
3,가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...
4,편의점 새내기입니다 선배님들께 여쭤봅니다! 이달25일부터 c사 편의점 인수예정인데...
...,...
12353,팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354,담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355,반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356,유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...


### 띄어쓰기 처리

In [23]:
def re_spacing(text):
    result = text.strip()
    while "  " in result:
        result = result.replace("  ", " ")
    return result

In [ ]:
doc['review'] = doc['review'].apply(lambda x : re_spacing(x))

In [27]:
doc
doc.reset_index(drop=True)

,review
0,편의점 마트 전기세 두배이상 올랐나요?? 기존에 가져가는것도 얼마 안되는데 마이너스...
1,5월 종합소득세 신고에서요 제가 사업자 신용카드로 발급 받아서 돈 쓴것도 지출 증빙...
2,사장님들 저가 sns로 소매로 판매하고있는데용 저가 초보라 잘몰라용 가루쳐주세요 ㅜ...
3,가게 오픈했습니다. 저랑 신랑이랑 둘이 할껀데요.. 작은 과일가게요.. 세무사기장료...
4,편의점 새내기입니다 선배님들께 여쭤봅니다! 이달25일부터 c사 편의점 인수예정인데...
...,...
12353,팔기싫은 손님 ㅠ 저는 소매업을 하고 있습니다 이제 장사를 좀 해서 그런지 촉이와요...
12354,담배반품 언제까지 받아주시나요? 구매한지 한달넘은 담배 들고와서 반품하겠다고 하네요...
12355,반품 할 것들은요 그냥 상온박스에 담아두면 되는건가요? 오셔서 일일이 체크하시려면 ...
12356,유통기한지난식품판매 안녕하세요~ 몇일전 부주의로 유통기한이 지난 라면2개를 팔게 되...


## CSV 파일로 저장

In [ ]:
doc.to_csv('[KB]아싸_전처리_final.csv', encoding='utf-8-sig')

In [ ]:
doc

,review
0,리모델링부터 시작한 신혼집 랜선집들이 저희는 구축 아파트를 매매해서 신혼집을 마련하...
1,행복한 고민 끝 결정했어요 네 가구가 가장 고민이 많았습니다 벽지 고를때 가장 힘든...
2,둘이아닌 셋 이제 셋이되고 달라진 저희집 거실과 침실 거실은 매트시공을 한상태로 나...
3,신혼집랜선집들이 저희 집을 소개합니다 결혼하고 이것저것 정리하고 거기에다 일하고 청...
4,베이지 취향 가득 오브제 정수기 소개합니다 저는 입주전 월에 전자에서 가전 세트로 ...
...,...
16180,인테리어 전후 홈씨씨인테리어에서했어요 날씨가 많이 추운데 예신님들 감기 조심하세요 ...
16181,햅삐한 노루네 신혼집 랜선집들이 초대합니다 신축 아파트 평 저희는 설 지나자마자 결...
16182,신혼집 인테리어 후기 입니다 저는 월말에 신혼집 오피스텔 에 먼저 입주해서 살고 있...
16183,아버지 친구분 수원 제일인테리어 께 시공받은 인테리어 후기 한샘 인테리어 후기 올려...
